In [2]:
import numpy as np
import scipy
import utils
from time import time
from utils import QR_Factorization, EVD, SVD, Bidiagonal_fastMult

%load_ext autoreload
%autoreload 2

# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [3]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


## Phase-II

In [4]:
m = n = 1024
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

Numpy @ might use multi-thread to accelerate the computation. But our implementation is O(n^2) which is theoretically more effcient.

In [5]:
B, _, _ = SVD.svd_phaseI(A)

Test for `fastMult_upper_bidiagonal`

In [6]:
numpy_mul_begin = time()
for i in range(1000):
    A@B
numpy_mul_end = time()
print("{:.4f}s".format(numpy_mul_end - numpy_mul_begin))

17.9793s


In [7]:
fastMul_begin = time()
for i in range(1000):
    Bidiagonal_fastMult.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
print("{:.4f}s".format(fastMul_end - fastMul_begin))

22.5654s


Test for `upper_fastMult_lower_bidiagonal`


In [8]:
numpy_mul_begin = time()
for i in range(1000):
    B@B.T
numpy_mul_end = time()
print("{:.4f}s".format(numpy_mul_end - numpy_mul_begin))

13.3726s


In [9]:
fastMul_begin = time()
for i in range(1000):
    Bidiagonal_fastMult.upper_fastMult_lower_bidiagonal(B, B.T)
fastMul_end = time()
print("{:.4f}s".format(fastMul_end - fastMul_begin))

7.6234s


Test for `qr_tridiagonal_by_Givens` and `qr_lower_bidiagonal_by_Givens`

In [10]:
begin = time()
for i in range(100):
    QR_Factorization.qr_tridiagonal_by_Givens(B.T, return_Givens=True)
end = time()
print("{:.4f}s".format(end-begin))

3.6122s


In [11]:
begin = time()
for i in range(100):
    QR_Factorization.qr_lower_bidiagonal_by_Givens(B.T, return_Givens=True)
end = time()
print("{:.4f}s".format(end-begin))

0.9907s


**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [12]:
m = 150
n = 150
A = np.random.rand(m,n)
A[n-50:n] = A[n-50:n] * 1000

In [13]:
U, S, Vt = SVD.svd(A, phaseII='A')
# U, S, Vt = SVD.svd(A, phaseII='A2')
# U, S, Vt = SVD.svd(A, phaseII='B')
# U, S, Vt = SVD.svd(A, phaseII='B2')
# U, S, Vt = SVD.svd(A, phaseII='C')
_, Ss, _  = scipy.linalg.svd(A, full_matrices=False)

SVD.accuracy_test(A, U, S, Vt, acc=1e-8)

phaseI: 0.0264s
phaseII: 0.4292s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
1.3774759111129242e-11


**Accuracy Test:**

Modify acc as you like!

In [14]:
SVD.accuracy_test(A, U, S, Vt, acc=1e-8)

Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
1.3774759111129242e-11


**Scipy SVD**

In [15]:
U, Ss, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-8
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(Ss)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %


## Test for p2

In [16]:
m = n = 100
k = 4
A = np.array(np.diag([1/2]*n, 0)+np.diag([1/4]*(n-1), 1)+np.diag([1/4]*(n-1), -1))

A = np.linalg.matrix_power(A, k)
U, S, Vt = SVD.svd(A, phaseII="B2")
_, Ss, _  = scipy.linalg.svd(A, full_matrices=False)

SVD.accuracy_test(A, U, S, Vt, acc=1e-8)

phaseI: 0.0143s
phaseII: 0.8226s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
8.584226131177716e-09


Modify acc as you like!

In [17]:
SVD.accuracy_test(A, U, S, Vt, acc=1e-8)

Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
8.584226131177716e-09


## Codes Sanity Check

In [148]:
m = n = 100
A = np.random.rand(m,n)
k = 5
B, Qt, P = SVD.svd_phaseI(A)

### Phase I check: 
Qt, P orthogonal.

In [135]:
SVD.is_orthogonal(Qt), SVD.is_orthogonal(P)

(True, True)

### Phase II Check

#### Phase II-A

In [136]:
T, G = EVD.eigh_by_QR(B@B.T, tol=1e-8)

In [137]:
SVD.is_orthogonal(G, silence=False)

2.021497192100585e-13


True

In [138]:
U, S, Vt = SVD.svd_phaseII(B, Qt, P, phaseII='A', eigen=EVD.eigh_by_QR)
SVD.is_orthogonal(U, silence=False), SVD.is_orthogonal(Vt.T, silence=False)

In [139]:
SVD.is_orthogonal(U, silence=False), SVD.is_orthogonal(Vt.T, silence=False)

2.1462127511422398e-13
3.988951347979838e-07


(True, True)

#### Phase II-B

In [153]:
U, S, Vt = SVD.svd_phaseII(B, Qt, P, phaseII='B1', eigen=EVD.eigh_of_BBT)
SVD.is_orthogonal(U, silence=False), SVD.is_orthogonal(Vt.T, silence=False)

2.666611720788574
146142.73800537895


(False, False)

In [143]:
U, S, Vt = SVD.svd_phaseII(B, Qt, P, phaseII='B2', eigen=EVD.eigh_of_BBT)
SVD.is_orthogonal(U, silence=False), SVD.is_orthogonal(Vt.T, silence=False)

2.412708597906401
179944.50199043343


(False, False)